<a href="https://colab.research.google.com/github/Naydhurve3/Neural-Network-SMS-Text-Classifier/blob/main/Copy_of_copy_of_fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Read data
train_dataset = pd.read_csv(
                            'train-data.tsv',sep='\t',
                            header=None,
                            names=['class', 'text']
                            )
valid_dataset = pd.read_csv(
                            'valid-data.tsv',
                             sep='\t',
                             header=None,
                             names=['class', 'text']
                             )

In [ ]:
# Preprocess data
def preprocess(df):
    data = df.copy()
    data.loc[data['class'] == 'ham','class'] = 0
    data.loc[data['class'] == 'spam','class'] = 1
    labels = data.pop('class').astype(np.float32)
    data = data.values
    return data, labels

In [ ]:
train_data, train_labels = preprocess(train_dataset)
valid_data, valid_labels = preprocess(valid_dataset)

In [ ]:
# Map text features to integer sequences
encoder = keras.layers.TextVectorization()
encoder.adapt(train_data)

In [ ]:
# Create RNN model
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
# Stopping condition
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                  mode="min",
                                  patience=3,
                                  restore_best_weights=True)

history = model.fit(x=train_data,y=train_labels, epochs=15,
                    validation_data=(valid_data,valid_labels),
                    callbacks=[callback]
                    )

In [ ]:
def predict_message(pred_text):
    # Preprocess the text using the encoder
    pred_text = np.array([pred_text])  # Convert to array and add batch dimension
    pred_text = tf.convert_to_tensor(pred_text, dtype=tf.string)  # Convert to Tensor

    # Ensure the shape is appropriate for TextVectorization
    pred_text = tf.squeeze(pred_text, axis=0)  # Remove the batch dimension

    # Predict with the model
    outcome = model.predict(tf.expand_dims(pred_text, axis=0))

    # Determine the tag (ham or spam)
    tag = 'ham'
    if outcome[0][0] > 0.5:
        tag = 'spam'

    prediction = [float(outcome[0][0]), tag]  # Convert outcome to float for safe JSON serialization

    return prediction

pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
